In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train
from src.models.train_model import train_conf_pen
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
# no normalization, using n != 0 and m!= 0 instead
# doesnt work, just use normalization instead
# dataset = Planetoid(root='data/Planetoid', name='Cora')

data = dataset[0].to(device)
data = random_splits(data, 50, 20)
splits = data.train_mask, data.val_mask, data.test_mask

print('-------------------------------------------------------------')
print(f'train size: {data.train_mask.sum()}')
print(f'valid size: {data.valid_mask.sum()}')
print(f'test size: {data.test_mask.sum()}')
print('-------------------------------------------------------------')

metrics = []

for preg_umask_mask_ratio in range(11):
    preg_umask_mask_ratio = preg_umask_mask_ratio / 10
    torch.manual_seed(1)
    random.seed(1)

    add_to_mask = set(random.sample(data.train_mask.numpy(), k=preg_umask_mask_ratio))
    preg_mask = torch.zeros(len(data.y), dtype=torch.bool)
    for i in add_to_mask:
        train_mask[i] = True

    gcn_model = GCN0(num_node_features=dataset.num_node_features,
                        num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train(gcn_model, data, beta=0, num_epochs=350, preg_mask = preg_mask)
    # gcn_model = train2(gcn_model, data)
    acc = evaluate0(gcn_model, data)
    # print(f'mu: {mu}, reg, Accuracy: {acc:.4f}')

    train_acc, val_acc, test_acc = evaluate1(gcn_model, data)
    metrics.append({'preg_umask_mask_ratio': preg_umask_mask_ratio, 'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc})
    print(f'preg_umask_mask_ratio: {preg_umask_mask_ratio}, train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}, test_acc: {test_acc:.4f}')
    print('-------------------------------------------------------------')



-------------------------------------------------------------
train size: 350
valid size: 140
test size: 2218
-------------------------------------------------------------


TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [5]:
print(data.train_mask.numpy())

[False False False ... False False False]
